# Purpose

Loading data from Bigquery to local was slow through pandas..     
Export the data to gcs and download data

In [1]:
from google.datalab import Context
import google.datalab.bigquery as bq
import google.datalab.storage as storage
import numpy as np
import pandas as pd
import json
from sklearn.metrics import f1_score
from io import StringIO

In [2]:
%load_ext google.datalab.kernel


In [4]:
%%bq query -n trips_q
#SQL query to return total births by year
WITH
  departing AS (
      SELECT
        DATE(starttime) date,
        EXTRACT(HOUR
        FROM
          starttime) hour,
        start_station_id station_id,
        COUNT(*) AS departing_bikes
      FROM
        `np-training.ableto.citibike`
      GROUP BY
        date,
        hour,
        station_id
    ),
  
  incoming AS(
      SELECT
        DATE(stoptime) date,
        EXTRACT(HOUR
        FROM
          stoptime) hour,
        end_station_id station_id,
        COUNT(*) AS incoming_bikes
      FROM
        `np-training.ableto.citibike`
      GROUP BY
        date,
        hour,
        station_id 
    )
SELECT
  COALESCE(d.date,
    i.date) AS date,
  COALESCE(d.hour,
    i.hour) AS hour,
  COALESCE(d.station_id,
    i.station_id) AS station_id,
  IFNULL(d.departing_bikes,
    0) AS departing_bikes,
  IFNULL(i.incoming_bikes,
    0) AS incoming_bikes
FROM
  departing d
FULL OUTER JOIN
  incoming i
ON
  d.date = i.date
  AND d.hour = i.hour
  AND d.station_id = i.station_id

In [5]:
%%bq execute -q trips_q -t ableto.citibike_daily -m overwrite


date,hour,station_id,departing_bikes,incoming_bikes
2017-04-13,0,3180,23,26
2016-09-25,0,380,21,3
2015-08-12,0,285,22,5
2017-07-22,0,368,28,12
2016-09-09,0,368,19,5
2017-06-25,0,387,25,13
2016-09-04,0,217,19,9
2015-09-25,0,521,24,6
2017-07-04,0,426,26,13
2016-11-15,0,379,20,2


**Save Data to GCS**     
Export the aggregated data to GCS

In [9]:
project_id = Context.default().project_id
bucket_path = 'gs://' + project_id
bucket_object = bucket_path + '/data/citibike/trips_daily.csv.gz'

table = bq.Table('ableto.citibike_daily')
table.extract(destination = bucket_object, csv_header=True,compress=True)

Job np-training/job_c3bxplcRqObyKEoPUehbx0Cfpta0 completed

In [10]:
%%bash -s "$bucket_object" "trips_daily.csv.gz"
gsutil cp $1 $2

Copying gs://np-training/data/citibike/trips_daily.csv.gz...
\ [1 files][ 35.9 MiB/ 35.9 MiB]                                                
Operation completed over 1 objects/35.9 MiB.                                     
